<a href="https://colab.research.google.com/github/OskarKrafft/Machine-Learning-Project/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [128]:
# Install packages and import them

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd
import pyreadstat

## Set up WD

In [129]:
# Mount GDrive to facilitate data import
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [130]:
# Change working directory to project folder
%cd /content/drive/MyDrive/Colab Notebooks/Machine-Learning-Project

/content/drive/MyDrive/Colab Notebooks/Machine-Learning-Project


In [131]:
# If Github repo is not yet cloned in Drive, use following code (just do once!):

# ! git clone https://github.com/OskarKrafft/Machine-Learning-Project.git

## Import data

In [81]:
eppes, meta = pyreadstat.read_sav('data/raw/1.0-EPPES.sav')
pd.set_option('display.max_columns', None)
eppes

,studyno1,studyno2,doi,version,edition,survey,caseid,uniqid,serialid,tnscntry,country,isocntry,split,q1.1,q1.2,q1.3,q1.4,q1.5,q1.6,q1.7,q1.8,q1.9,q1.10,q1.11,q1.12,q1.13,q1.14,q1.15,q1.16,q1.17,q1.18,q1.19,q1.20,q1.21,q1.22,q1.23,q1.24,q1.25,q1.26,q1.27,q1.28,q1.29,q1.30,q1.31,q1.32,q1.33,q1.34,q1.35,q1.36,d70,d71a_1,d71a_2,d71a_3,d71b_1,d71b_2,d71b_3,opls,polintr,qa1a_1,qa1a_2,qa1a_3,qa1a_4,qa1a_5,qa1a_6,qa1a_7,qa1b_1,qa1b_2,qa1b_3,qa1b_4,qa1b_5,qa1b_6,qa1b_7,qa2a_1,qa2a_2,qa2a_3,qa2a_4,qa2a_5,qa2a_6,qa2a_7,qa2b_1,qa2b_2,qa2b_3,qa2b_4,qa2b_5,qa2b_6,qa2b_7,qa3a.1,qa3a.2,qa3a.3,qa3a.4,qa3a.5,qa3a.6,qa3a.7,qa3a.8,qa3a.9,qa3a.10,qa3a.11,qa3a.12,qa3a.13,qa3a.14,qa3a.15,qa3a.16,qa4a.1,qa4a.2,qa4a.3,qa4a.4,qa4a.5,qa4a.6,qa4a.7,qa4a.8,qa4a.9,qa4a.10,qa4a.11,qa4a.12,qa4a.13,qa4a.14,qa4a.15,qa4a.16,qa4a.17,qa4a.18,qa3b.1,qa3b.2,qa3b.3,qa3b.4,qa3b.5,qa3b.6,qa3b.7,qa3b.8,qa3b.9,qa3b.10,qa3b.11,qa3b.12,qa3b.13,qa3b.14,qa3b.15,qa3b.16,qa4b.1,qa4b.2,qa4b.3,qa4b.4,qa4b.5,qa4b.6,qa4b.7,qa4b.8,qa4b.9,qa4b.10,qa4b.11,qa4b.12,qa4b.13,qa4b.14,qa4b.15,qa4b.16,qa4b.17,qa4b.18,qa5.1,qa5.2,qa5.3,qa5.4,qa5.5,qa5.6,qa5.7,qa5.8,qa5.9,qa5.10,qa5.11,qa5.12,qa5.13,qa5.14,qa5.15,qa5.16,d73a_1,d73a_2,d73a_3,d73b_1,d73b_2,d73b_3,qa6a_1,qa6a_2,qa6a_3,qa6a_4,qa6a_5,qa6a_6,qa6a_7,qa6a_8,qa6a_9,qa6a_10,qa6a_11,qa6b_1,qa6b_2,qa6b_3,qa6b_4,qa7,qa8_1,qa8_2,qa8_3,qa8_4,qa8_5,qa8_6,qa8_7,qa8_8,qa8_9,qa8_10,qa8_11,qa8_12,qa8_13,qa9.1,qa9.2,qa9.3,qa9.4,qa9.5,qa9.6,qa9.7,qa9.8,qa9.9,qa9.10,qa9.11,qa9.12,qa9.13,qa9.14,qa9.15,qa9.16,qa10_1,qa10_2,qa10_3,qa10_4,qa10_5,qa10_6,qf1,qf2,qf3.1,qf3.2,qf3.3,qf3.4,qf3.5,qf3.6,qf3.7,qf3.8,qf3.9,qf3.10,qf3.11,qf3.12,qf3.13,qf3.14,qf3.15,qa11a,qa12a,qa11b,qa12b,qa13_1,qa13_2,qa13_3,qa13_4,qa13_5,qa14_1,qa14_2,qa14_3,qa14_4,qa14_5,qa15_1,qa15_2,qa15_3,qa15t1,qa15t2,qa15t3,qa15t4,qa15t5,qa15t6,qa15t7,qa16_1,qa16_2,qa16_3,qa16_4,qa16_5,qa16_6,qa16_7,qa16_8,qa16_9,qa16r,qa17a,qa17b,qa18a_1,qa18a_2,qa18a_3,qa18a_4,qa18a_5,qa18a_6,qa18b_1,qa18b_2,qa18b_3,qa19,qb1_1,qb2.1,qb2.2,qb2.3,qb2.4,qb2.5,qb2.6,qb2.7,qb2.8,qb2.9,qb2.10,qb2.11,qb2.12,qb2.13,qb2.14,qb3_1,qb3_2,qb4,qb5_1,qb5_2,qb5_3,qb5_4,qc1,qc2a_1,qc2a_2,qc2a_3,qc2a_4,qc2a_5,qc2b_1,qc2b_2,qd1a_1,qd1a_2,qd1a_3,qd1a_4,qd1b_1,qd1b_2,qd1b_3,qd1b_4,qd2_1,qd2_2,qd2_3,qd3,qd4a,qd4b.1,qd4b.2,qd4b.3,qd4b.4,qd4b.5,qd4b.6,qd4b.7,qd4b.8,qd4b.9,qd4b.10,qd4b.11,qd4t.1,qd4t.2,qd4t.3,qd4t.4,qd4t.5,qd4t.6,qd4t.7,qd4t.8,qd4t.9,qd4t.10,qd4t.11,qd5.1,qd5.2,qd5.3,qd5.4,qd5.5,qd5.6,qd5.7,qd5.8,qd5.9,qd5.10,qd5.11,qd5.12,qd5.13,qd5.14,qd5.15,qd5.16,qd5.17,qd6.1,qd6.2,qd6.3,qd6.4,qd6.5,qd6.6,qd6.7,qd6.8,qd6.9,qd6.10,qd6.11,qd6.12,qd6.13,qd6.14,qd7.1,qd7.2,qd7.3,qd7.4,qd7.5,qd7.6,qd7.7,qd7.8,qd7.9,qd7.10,qd7.11,qd7.12,qd7.13,qd7.14,qd8,qd9_1,qd9_2,qd9_3,qd9_4,qd9_5,qd9_6,qd9_7,qd10a,qd10ar,qd10b,qd10br,qd11_1,qd11_2,qd11_3,qd11_4,qd11_5,qd11_6,qd11_7,qd11_8,qd12_1,qd12_2,qe1_1,qe1_2,qe1_3,qe1_4,qe1_5,qe1_6,qe1_7,qe1_8,qe2_1,qe2_2,qe2_3,qe2_4,qg1,qg1_orig,qg1b,qg2,qg3,qg4a,qg4b.1,qg4b.2,qg4b.3,qg4b.4,qg4b.5,qg4b.6,qg4b.7,qg4b.8,qg4b.9,qg4t.1,qg4t.2,qg4t.3,qg4t.4,qg4t.5,qg4t.6,qg4t.7,qg4t.8,qg4t.9,qg5,qg6.1,qg6.2,qg6.3,qg6.4,qg6.5,qg6.6,qg6.7,qg6.8,qg6.9,qg6.10,qg6.11,qg6.12,qg6.13,qg6.14,qg6.15,qg6.16,qg6.17,qg7a,qg7b.1,qg7b.2,qg7b.3,qg7b.4,qg7b.5,qg7b.6,qg7b.7,qg7b.8,qg7b.9,qg7b.10,qg7b.11,qg7b.12,qg7b.13,qg7b.14,qg7b.15,qg7b.16,qg7t.1,qg7t.2,qg7t.3,qg7t.4,qg7t.5,qg7t.6,qg7t.7,qg7t.8,qg7t.9,qg7t.10,qg7t.11,qg7t.12,qg7t.13,qg7t.14,qg7t.15,qg7t.16,qg8,qg9.1,qg9.2,qg9.3,qg9.4,qg9.5,qg9.6,qg9.7,qg9.8,qg9.9,qg9.10,qg9.11,qg9.12,qg9.13,qg9.14,qg9.15,qg9.16,qg9.17,qg10,qg11_1,qg11_2,qg11_3,qg11_4,qg11_5,qg11_6,qg11_7,qg11_8,qg11_9,qg12,qg13_1,qg13_2,qg13_3,qg13_4,qg13_5,qg13_5v,qg14_1,qg14_2,qg14_1v,qg14_2v,qg15,qg16,qh1,qh2.1,qh2.2,qh2.3,qh2.4,qh2.5,qh2.6,qh2.7,qh2.8,qh2.9,qh2.10,qh2.11,qh2.12,qh2.13,qh2.14,qh2.15,qh2.16,qh3,qh4,qh5_1,qh5_2,qh5_3,qh5_4,qh5_5,qh5_6,qh5_7,qh6,d1,d1r1,d1r2,d7,d7r1,d7r2,d7r3,d8,d8r1,d8r2,d10,d11,d11r1,d11r2,d11r3,gen1,gen2,gen3,gen4,gen5,gen6,d15a,d15a_r1,d15a_r2,d15b,d15b_r,d25,d40a,d4

# Data Cleaning

## Drop non-voters

In [82]:
# Drop observation of people that were not eligible to vote

eppes = eppes[eppes.qg1.isna() == False]
eppes

,studyno1,studyno2,doi,version,edition,survey,caseid,uniqid,serialid,tnscntry,country,isocntry,split,q1.1,q1.2,q1.3,q1.4,q1.5,q1.6,q1.7,q1.8,q1.9,q1.10,q1.11,q1.12,q1.13,q1.14,q1.15,q1.16,q1.17,q1.18,q1.19,q1.20,q1.21,q1.22,q1.23,q1.24,q1.25,q1.26,q1.27,q1.28,q1.29,q1.30,q1.31,q1.32,q1.33,q1.34,q1.35,q1.36,d70,d71a_1,d71a_2,d71a_3,d71b_1,d71b_2,d71b_3,opls,polintr,qa1a_1,qa1a_2,qa1a_3,qa1a_4,qa1a_5,qa1a_6,qa1a_7,qa1b_1,qa1b_2,qa1b_3,qa1b_4,qa1b_5,qa1b_6,qa1b_7,qa2a_1,qa2a_2,qa2a_3,qa2a_4,qa2a_5,qa2a_6,qa2a_7,qa2b_1,qa2b_2,qa2b_3,qa2b_4,qa2b_5,qa2b_6,qa2b_7,qa3a.1,qa3a.2,qa3a.3,qa3a.4,qa3a.5,qa3a.6,qa3a.7,qa3a.8,qa3a.9,qa3a.10,qa3a.11,qa3a.12,qa3a.13,qa3a.14,qa3a.15,qa3a.16,qa4a.1,qa4a.2,qa4a.3,qa4a.4,qa4a.5,qa4a.6,qa4a.7,qa4a.8,qa4a.9,qa4a.10,qa4a.11,qa4a.12,qa4a.13,qa4a.14,qa4a.15,qa4a.16,qa4a.17,qa4a.18,qa3b.1,qa3b.2,qa3b.3,qa3b.4,qa3b.5,qa3b.6,qa3b.7,qa3b.8,qa3b.9,qa3b.10,qa3b.11,qa3b.12,qa3b.13,qa3b.14,qa3b.15,qa3b.16,qa4b.1,qa4b.2,qa4b.3,qa4b.4,qa4b.5,qa4b.6,qa4b.7,qa4b.8,qa4b.9,qa4b.10,qa4b.11,qa4b.12,qa4b.13,qa4b.14,qa4b.15,qa4b.16,qa4b.17,qa4b.18,qa5.1,qa5.2,qa5.3,qa5.4,qa5.5,qa5.6,qa5.7,qa5.8,qa5.9,qa5.10,qa5.11,qa5.12,qa5.13,qa5.14,qa5.15,qa5.16,d73a_1,d73a_2,d73a_3,d73b_1,d73b_2,d73b_3,qa6a_1,qa6a_2,qa6a_3,qa6a_4,qa6a_5,qa6a_6,qa6a_7,qa6a_8,qa6a_9,qa6a_10,qa6a_11,qa6b_1,qa6b_2,qa6b_3,qa6b_4,qa7,qa8_1,qa8_2,qa8_3,qa8_4,qa8_5,qa8_6,qa8_7,qa8_8,qa8_9,qa8_10,qa8_11,qa8_12,qa8_13,qa9.1,qa9.2,qa9.3,qa9.4,qa9.5,qa9.6,qa9.7,qa9.8,qa9.9,qa9.10,qa9.11,qa9.12,qa9.13,qa9.14,qa9.15,qa9.16,qa10_1,qa10_2,qa10_3,qa10_4,qa10_5,qa10_6,qf1,qf2,qf3.1,qf3.2,qf3.3,qf3.4,qf3.5,qf3.6,qf3.7,qf3.8,qf3.9,qf3.10,qf3.11,qf3.12,qf3.13,qf3.14,qf3.15,qa11a,qa12a,qa11b,qa12b,qa13_1,qa13_2,qa13_3,qa13_4,qa13_5,qa14_1,qa14_2,qa14_3,qa14_4,qa14_5,qa15_1,qa15_2,qa15_3,qa15t1,qa15t2,qa15t3,qa15t4,qa15t5,qa15t6,qa15t7,qa16_1,qa16_2,qa16_3,qa16_4,qa16_5,qa16_6,qa16_7,qa16_8,qa16_9,qa16r,qa17a,qa17b,qa18a_1,qa18a_2,qa18a_3,qa18a_4,qa18a_5,qa18a_6,qa18b_1,qa18b_2,qa18b_3,qa19,qb1_1,qb2.1,qb2.2,qb2.3,qb2.4,qb2.5,qb2.6,qb2.7,qb2.8,qb2.9,qb2.10,qb2.11,qb2.12,qb2.13,qb2.14,qb3_1,qb3_2,qb4,qb5_1,qb5_2,qb5_3,qb5_4,qc1,qc2a_1,qc2a_2,qc2a_3,qc2a_4,qc2a_5,qc2b_1,qc2b_2,qd1a_1,qd1a_2,qd1a_3,qd1a_4,qd1b_1,qd1b_2,qd1b_3,qd1b_4,qd2_1,qd2_2,qd2_3,qd3,qd4a,qd4b.1,qd4b.2,qd4b.3,qd4b.4,qd4b.5,qd4b.6,qd4b.7,qd4b.8,qd4b.9,qd4b.10,qd4b.11,qd4t.1,qd4t.2,qd4t.3,qd4t.4,qd4t.5,qd4t.6,qd4t.7,qd4t.8,qd4t.9,qd4t.10,qd4t.11,qd5.1,qd5.2,qd5.3,qd5.4,qd5.5,qd5.6,qd5.7,qd5.8,qd5.9,qd5.10,qd5.11,qd5.12,qd5.13,qd5.14,qd5.15,qd5.16,qd5.17,qd6.1,qd6.2,qd6.3,qd6.4,qd6.5,qd6.6,qd6.7,qd6.8,qd6.9,qd6.10,qd6.11,qd6.12,qd6.13,qd6.14,qd7.1,qd7.2,qd7.3,qd7.4,qd7.5,qd7.6,qd7.7,qd7.8,qd7.9,qd7.10,qd7.11,qd7.12,qd7.13,qd7.14,qd8,qd9_1,qd9_2,qd9_3,qd9_4,qd9_5,qd9_6,qd9_7,qd10a,qd10ar,qd10b,qd10br,qd11_1,qd11_2,qd11_3,qd11_4,qd11_5,qd11_6,qd11_7,qd11_8,qd12_1,qd12_2,qe1_1,qe1_2,qe1_3,qe1_4,qe1_5,qe1_6,qe1_7,qe1_8,qe2_1,qe2_2,qe2_3,qe2_4,qg1,qg1_orig,qg1b,qg2,qg3,qg4a,qg4b.1,qg4b.2,qg4b.3,qg4b.4,qg4b.5,qg4b.6,qg4b.7,qg4b.8,qg4b.9,qg4t.1,qg4t.2,qg4t.3,qg4t.4,qg4t.5,qg4t.6,qg4t.7,qg4t.8,qg4t.9,qg5,qg6.1,qg6.2,qg6.3,qg6.4,qg6.5,qg6.6,qg6.7,qg6.8,qg6.9,qg6.10,qg6.11,qg6.12,qg6.13,qg6.14,qg6.15,qg6.16,qg6.17,qg7a,qg7b.1,qg7b.2,qg7b.3,qg7b.4,qg7b.5,qg7b.6,qg7b.7,qg7b.8,qg7b.9,qg7b.10,qg7b.11,qg7b.12,qg7b.13,qg7b.14,qg7b.15,qg7b.16,qg7t.1,qg7t.2,qg7t.3,qg7t.4,qg7t.5,qg7t.6,qg7t.7,qg7t.8,qg7t.9,qg7t.10,qg7t.11,qg7t.12,qg7t.13,qg7t.14,qg7t.15,qg7t.16,qg8,qg9.1,qg9.2,qg9.3,qg9.4,qg9.5,qg9.6,qg9.7,qg9.8,qg9.9,qg9.10,qg9.11,qg9.12,qg9.13,qg9.14,qg9.15,qg9.16,qg9.17,qg10,qg11_1,qg11_2,qg11_3,qg11_4,qg11_5,qg11_6,qg11_7,qg11_8,qg11_9,qg12,qg13_1,qg13_2,qg13_3,qg13_4,qg13_5,qg13_5v,qg14_1,qg14_2,qg14_1v,qg14_2v,qg15,qg16,qh1,qh2.1,qh2.2,qh2.3,qh2.4,qh2.5,qh2.6,qh2.7,qh2.8,qh2.9,qh2.10,qh2.11,qh2.12,qh2.13,qh2.14,qh2.15,qh2.16,qh3,qh4,qh5_1,qh5_2,qh5_3,qh5_4,qh5_5,qh5_6,qh5_7,qh6,d1,d1r1,d1r2,d7,d7r1,d7r2,d7r3,d8,d8r1,d8r2,d10,d11,d11r1,d11r2,d11r3,gen1,gen2,gen3,gen4,gen5,gen6,d15a,d15a_r1,d15a_r2,d15b,d15b_r,d25,d40a,d4

In [83]:
# Check for NAs in vote variable qg1

eppes['qg1'].isna().sum()

0

In [51]:
# Create frequency table of qg1

eppes["qg1"].value_counts()

1.0    15008
2.0    12456
Name: qg1, dtype: int64

## Drop problematic, non-complete or uninteresting columns

In [85]:
# Import Excel sheet containing column indeces to be dropped

columns_analysis = pd.read_excel(r'Drop_Columns.xlsx')
columns_analysis = columns_analysis.drop(columns_analysis.columns[[0]], axis = 1)
print(columns_analysis)

     col_index col_names  keep_columns encoding
0            0  studyno1          True       no
1            1  studyno2          True       no
2            2       doi          True       no
3            3   version          True       no
4            4   edition          True       no
..         ...       ...           ...      ...
867        867      w140         False       no
868        868      w144         False       no
869        869      w148         False       no
870        870      w149         False       no
871        871   wexmen2         False       no

[872 rows x 4 columns]


In [73]:
# Create list of indeces of columns to be dropped

indeces_to_drop = []

for i in range(872):
  if columns_analysis.iloc[i, 2] == False:
    indeces_to_drop.append(i)

print(indeces_to_drop)

[12, 41, 42, 43, 44, 45, 46, 47, 48, 53, 54, 55, 56, 57, 65, 66, 67, 68, 69, 70, 71, 79, 80, 81, 82, 83, 84, 85, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 173, 174, 175, 187, 188, 189, 190, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 261, 262, 263, 264, 265, 266, 267, 277, 286, 287, 288, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 307, 318, 319, 324, 325, 326, 327, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 411, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 49

In [84]:
# Drop columns by indeces_to_drop

eppes = eppes.drop(eppes.columns[indeces_to_drop], axis = 1)
eppes

,studyno1,studyno2,doi,version,edition,survey,caseid,uniqid,serialid,tnscntry,country,isocntry,q1.1,q1.2,q1.3,q1.4,q1.5,q1.6,q1.7,q1.8,q1.9,q1.10,q1.11,q1.12,q1.13,q1.14,q1.15,q1.16,q1.17,q1.18,q1.19,q1.20,q1.21,q1.22,q1.23,q1.24,q1.25,q1.26,q1.27,q1.28,d70,d71a_1,d71a_2,d71a_3,qa1a_1,qa1a_2,qa1a_3,qa1a_4,qa1a_5,qa1a_6,qa1a_7,qa2a_1,qa2a_2,qa2a_3,qa2a_4,qa2a_5,qa2a_6,qa2a_7,qa3a.1,qa3a.2,qa3a.3,qa3a.4,qa3a.5,qa3a.6,qa3a.7,qa3a.8,qa3a.9,qa3a.10,qa3a.11,qa3a.12,qa3a.13,qa3a.14,qa3a.15,qa3a.16,qa4a.1,qa4a.2,qa4a.3,qa4a.4,qa4a.5,qa4a.6,qa4a.7,qa4a.8,qa4a.9,qa4a.10,qa4a.11,qa4a.12,qa4a.13,qa4a.14,qa4a.15,qa4a.16,qa4a.17,qa4a.18,qa5.1,qa5.2,qa5.3,qa5.4,qa5.5,qa5.6,qa5.7,qa5.8,qa5.9,qa5.10,qa5.11,qa5.12,qa5.13,qa5.14,qa5.15,qa5.16,d73a_1,d73a_2,d73a_3,qa6a_1,qa6a_2,qa6a_3,qa6a_4,qa6a_5,qa6a_6,qa6a_7,qa6a_8,qa6a_9,qa6a_10,qa6a_11,qa7,qa8_1,qa8_2,qa8_3,qa8_4,qa8_5,qa8_6,qa8_7,qa8_8,qa8_9,qa8_10,qa8_11,qa8_12,qa8_13,qa9.1,qa9.2,qa9.3,qa9.4,qa9.5,qa9.6,qa9.7,qa9.8,qa9.9,qa9.10,qa9.11,qa9.12,qa9.13,qa9.14,qa9.15,qa9.16,qa10_1,qa10_2,qa10_3,qa10_4,qa10_5,qa10_6,qf1,qf2,qa13_1,qa13_2,qa13_3,qa13_4,qa13_5,qa14_1,qa14_2,qa14_3,qa14_4,qa14_5,qa15_1,qa15_2,qa15_3,qa16_1,qa16_2,qa16_3,qa16_4,qa16_5,qa16_6,qa16_7,qa16_8,qa16_9,qa17a,qa17b,qa18a_1,qa18a_2,qa18a_3,qa18a_4,qa18a_5,qa18a_6,qa19,qb3_1,qb3_2,qb5_1,qb5_2,qb5_3,qb5_4,qc1,qc2a_1,qc2a_2,qc2a_3,qc2a_4,qc2a_5,qd1a_1,qd1a_2,qd1a_3,qd1a_4,qd2_1,qd2_2,qd2_3,qd3,qd4a,qd5.1,qd5.2,qd5.3,qd5.4,qd5.5,qd5.6,qd5.7,qd5.8,qd5.9,qd5.10,qd5.11,qd5.12,qd5.13,qd5.14,qd5.15,qd5.16,qd5.17,qd6.1,qd6.2,qd6.3,qd6.4,qd6.5,qd6.6,qd6.7,qd6.8,qd6.9,qd6.10,qd6.11,qd6.12,qd6.13,qd6.14,qd7.1,qd7.2,qd7.3,qd7.4,qd7.5,qd7.6,qd7.7,qd7.8,qd7.9,qd7.10,qd7.11,qd7.12,qd7.13,qd7.14,qd8,qd9_1,qd9_2,qd9_3,qd9_4,qd9_5,qd9_6,qd9_7,qd10a,qd10ar,qd10b,qd11_1,qd11_2,qd11_3,qd11_4,qd11_5,qd11_6,qd11_7,qd11_8,qd12_1,qd12_2,qe1_1,qe1_2,qe1_3,qe1_4,qe1_5,qe1_6,qe1_7,qe1_8,qe2_1,qe2_2,qe2_3,qe2_4,qg1,qg10,qg11_1,qg11_2,qg11_3,qg11_4,qg11_5,qg11_6,qg11_7,qg11_8,qg11_9,qg12,qg13_1,qg13_2,qg13_3,qg13_4,qg13_5,qg15,d1,d7,d11,d15a,d25,d40a,d40b,d43a,d43b,d46.8,d60,d62_1,d62_2,d63,d72_1,d72_2,d77
0,7576.0,7576.0,doi:10.4232/1.13393,1.0.0 (2019-12-09),1.0,915.0,17.0,10000017.0,1.0,1.0,2.0,BE,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,4.0,4.0,3.0,2.0,3.0,2.0,2.0,4.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,3.0,2.0,2.0,3.0,3.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,2.0,2.0,3.0,3.0,3.0,3.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,NaN,2.0,2.0,2.0,3.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,7.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,5.0,5.0,4.0,4.0,2.0,1.0,2.0,28.0,1.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,3.0,6.0,1.0,3.0,3.0,2.0
1,7576.0,7576.0,doi:10.4232/1.13393,1.0.0 (2019-12-09),1.0,915.0,21.0,10000021.0,2.0,1.0,2.0,BE,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,3.0,1.0,1.0,3.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,

## Deal with NAs

In [32]:
# Check for NAs in columns
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  print(eppes.isna().sum())

studyno1        0
studyno2        0
doi             0
version         0
edition         0
survey          0
caseid          0
uniqid          0
serialid        0
tnscntry        0
country         0
isocntry        0
q1.1            0
q1.2            0
q1.3            0
q1.4            0
q1.5            0
q1.6            0
q1.7            0
q1.8            0
q1.9            0
q1.10           0
q1.11           0
q1.12           0
q1.13           0
q1.14           0
q1.15           0
q1.16           0
q1.17           0
q1.18           0
q1.19           0
q1.20           0
q1.21           0
q1.22           0
q1.23           0
q1.24           0
q1.25           0
q1.26           0
q1.27           0
q1.28           0
d70             0
d71a_1          0
d71a_2          0
d71a_3          0
qa1a_1          0
qa1a_2          0
qa1a_3          0
qa1a_4          0
qa1a_5          0
qa1a_6          0
qa1a_7          0
qa2a_1          0
qa2a_2          0
qa2a_3          0
qa2a_4          0
qa2a_5    

Result: qc2a_1 and qc2a_2, d1, d25, d40b contain NAs

Next Steps

1. **qc2a_1** and **qc2a_2**: 

For each of the following statements, please tell me whether you totally agree, tend to agree, tend to disagree or totally disagree.

qc2a_1: Measures to reduce the public deficit and debt in (OUR COUNTRY) cannot be delayed

qc2a_2: Measures to reduce the public deficit and debt in (OUR COUNTRY) are not a priority for now

For these two questions, the dataset was split into two groups (former columns index 12) and only group 1 was asked qc2a_1, only group 2 was asked qc2a_2

-> Drop qc2a_1 and qc2a_2 columns

2. **d1**: 

In political matters people talk of "the left" and "the right". How would you place your views on this scale?

Important Question

-> Change values of rows with d1 == NA to d1 == 12 (DK)

3. **d25**:

Would you say you live in a...? Rural area or village, small or middle sized town, etc.

-> Change values of rows with d25 == NA to d25 == 4 (DK)

4. **d40b**:

Could you tell me how many children less than 10 years old live in your household?

-> replace values with 0

In [55]:
# Drop qc2a_1 and qc2a_2

eppes = eppes.drop(['qc2a_1', 'qc2a_2'], axis=1)

# Replace values of rows with d1 == NA with d1 == 12

eppes.loc[eppes['d1'].isna(), 'd1'] = 12

# Replace values of rows with d25 == NA with d25 == 4

eppes.loc[eppes['d25'].isna(), 'd25'] = 4

# Replace values of rows with d40b == NA with d40b == 0

eppes.loc[eppes['d40b'].isna(), 'd40b'] = 0


## warum nicht die NAs bei d40b direkt droppen? sind im endeffekt ja auch nur 2 observations.

In [58]:
# Check for NAs in eppes
eppes.isna().sum().sum()

0

## Delete rows with NAs in ordinal variables

In [35]:
eppes.describe()

,studyno1,studyno2,edition,survey,caseid,uniqid,serialid,tnscntry,country,q1.1,q1.2,q1.3,q1.4,q1.5,q1.6,q1.7,q1.8,q1.9,q1.10,q1.11,q1.12,q1.13,q1.14,q1.15,q1.16,q1.17,q1.18,q1.19,q1.20,q1.21,q1.22,q1.23,q1.24,q1.25,q1.26,q1.27,q1.28,d70,d71a_1,d71a_2,d71a_3,qa1a_1,qa1a_2,qa1a_3,qa1a_4,qa1a_5,qa1a_6,qa1a_7,qa2a_1,qa2a_2,qa2a_3,qa2a_4,qa2a_5,qa2a_6,qa2a_7,qa3a.1,qa3a.2,qa3a.3,qa3a.4,qa3a.5,qa3a.6,qa3a.7,qa3a.8,qa3a.9,qa3a.10,qa3a.11,qa3a.12,qa3a.13,qa3a.14,qa3a.15,qa3a.16,qa4a.1,qa4a.2,qa4a.3,qa4a.4,qa4a.5,qa4a.6,qa4a.7,qa4a.8,qa4a.9,qa4a.10,qa4a.11,qa4a.12,qa4a.13,qa4a.14,qa4a.15,qa4a.16,qa4a.17,qa4a.18,qa5.1,qa5.2,qa5.3,qa5.4,qa5.5,qa5.6,qa5.7,qa5.8,qa5.9,qa5.10,qa5.11,qa5.12,qa5.13,qa5.14,qa5.15,qa5.16,d73a_1,d73a_2,d73a_3,qa6a_1,qa6a_2,qa6a_3,qa6a_4,qa6a_5,qa6a_6,qa6a_7,qa6a_8,qa6a_9,qa6a_10,qa6a_11,qa7,qa8_1,qa8_2,qa8_3,qa8_4,qa8_5,qa8_6,qa8_7,qa8_8,qa8_9,qa8_10,qa8_11,qa8_12,qa8_13,qa9.1,qa9.2,qa9.3,qa9.4,qa9.5,qa9.6,qa9.7,qa9.8,qa9.9,qa9.10,qa9.11,qa9.12,qa9.13,qa9.14,qa9.15,qa9.16,qa10_1,qa10_2,qa10_3,qa10_4,qa10_5,qa10_6,qf1,qf2,qa13_1,qa13_2,qa13_3,qa13_4,qa13_5,qa14_1,qa14_2,qa14_3,qa14_4,qa14_5,qa15_1,qa15_2,qa15_3,qa16_1,qa16_2,qa16_3,qa16_4,qa16_5,qa16_6,qa16_7,qa16_8,qa16_9,qa17a,qa17b,qa18a_1,qa18a_2,qa18a_3,qa18a_4,qa18a_5,qa18a_6,qa19,qb3_1,qb3_2,qb5_1,qb5_2,qb5_3,qb5_4,qc1,qc2a_3,qc2a_4,qc2a_5,qd1a_1,qd1a_2,qd1a_3,qd1a_4,qd2_1,qd2_2,qd2_3,qd3,qd4a,qd5.1,qd5.2,qd5.3,qd5.4,qd5.5,qd5.6,qd5.7,qd5.8,qd5.9,qd5.10,qd5.11,qd5.12,qd5.13,qd5.14,qd5.15,qd5.16,qd5.17,qd6.1,qd6.2,qd6.3,qd6.4,qd6.5,qd6.6,qd6.7,qd6.8,qd6.9,qd6.10,qd6.11,qd6.12,qd6.13,qd6.14,qd7.1,qd7.2,qd7.3,qd7.4,qd7.5,qd7.6,qd7.7,qd7.8,qd7.9,qd7.10,qd7.11,qd7.12,qd7.13,qd7.14,qd8,qd9_1,qd9_2,qd9_3,qd9_4,qd9_5,qd9_6,qd9_7,qd10a,qd10ar,qd10b,qd11_1,qd11_2,qd11_3,qd11_4,qd11_5,qd11_6,qd11_7,qd11_8,qd12_1,qd12_2,qe1_1,qe1_2,qe1_3,qe1_4,qe1_5,qe1_6,qe1_7,qe1_8,qe2_1,qe2_2,qe2_3,qe2_4,qg1,qg10,qg11_1,qg11_2,qg11_3,qg11_4,qg11_5,qg11_6,qg11_7,qg11_8,qg11_9,qg12,qg13_1,qg13_2,qg13_3,qg13_4,qg13_5,qg15,d1,d7,d11,d15a,d25,d40a,d40b,d43a,d43b,d46.8,d60,d62_1,d62_2,d63,d72_1,d72_2,d77
count,27464.0,27464.0,27464.0,27464.0,2.746400e+04,2.746400e+04,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.00000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.00000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.00000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,

d1, d7 and d11 have values of 97/98?

In [36]:
eppes["d1"].value_counts()

5.0     6563
12.0    2714
3.0     2489
4.0     2398
6.0     2380
97.0    2336
7.0     2184
8.0     1843
1.0     1434
10.0    1380
2.0     1060
9.0      683
Name: d1, dtype: int64

In [37]:
eppes["d7"].value_counts()

1.0     7318
2.0     6711
9.0     4135
13.0    2367
5.0     2209
11.0    1517
6.0      958
12.0     600
10.0     411
14.0     407
15.0     258
7.0      163
3.0      161
4.0      114
8.0       69
97.0      66
Name: d7, dtype: int64

In [38]:
eppes["d11"].value_counts()

65.0    605
68.0    603
55.0    570
60.0    567
50.0    549
       ... 
96.0      5
93.0      5
95.0      3
98.0      2
97.0      1
Name: d11, Length: 84, dtype: int64

Explanation: 

- d1 and d7: probably DKs
- d11: true age, no changes necessary
- for most variables: Highest number stands for DK


In [86]:
for i in range(86):
  print(eppes[col_names_ordinal[i]].value_counts())

2.0    15674
1.0     6943
3.0     3859
4.0      878
5.0      110
Name: d70, dtype: int64
2.0    15343
1.0     6143
3.0     5885
4.0       93
Name: d71a_1, dtype: int64
2.0    14981
3.0     8282
1.0     4047
4.0      154
Name: d71a_2, dtype: int64
2.0    15116
3.0     6473
1.0     5768
4.0      107
Name: d71a_3, dtype: int64
2.0    14008
3.0     8751
4.0     2198
1.0     1932
5.0      575
Name: qa1a_1, dtype: int64
2.0    12465
3.0     9057
4.0     2556
1.0     2490
5.0      896
Name: qa1a_2, dtype: int64
2.0    14316
3.0     6953
5.0     3550
1.0     1593
4.0     1052
Name: qa1a_3, dtype: int64
2.0    11745
5.0     5436
1.0     4696
3.0     4188
4.0     1399
Name: qa1a_4, dtype: int64
2.0    16003
3.0     5873
1.0     3685
4.0     1303
5.0      600
Name: qa1a_5, dtype: int64
2.0    11671
3.0     9031
4.0     2917
1.0     2640
5.0     1205
Name: qa1a_6, dtype: int64
2.0    13404
3.0     8226
1.0     2502
4.0     2126
5.0     1206
Name: qa1a_7, dtype: int64
3.0    16398
1.0     7700
2.0 

In [40]:
columns_analysis["encoding"].value_counts()

no             710
ordinal         88
categorical     74
Name: encoding, dtype: int64

In [98]:
# Create list of names of ordinal columns 

col_names_ordinal = []

for i in range(872):
  if columns_analysis.iloc[i, 3] == 'ordinal':
    col_names_ordinal.append(columns_analysis.iloc[i, 1])

# Delete qa1a_1 and qa1a_2 see above
col_names_ordinal.remove('qc2a_1')
col_names_ordinal.remove('qc2a_2')

print(col_names_ordinal)

['d70', 'd71a_1', 'd71a_2', 'd71a_3', 'qa1a_1', 'qa1a_2', 'qa1a_3', 'qa1a_4', 'qa1a_5', 'qa1a_6', 'qa1a_7', 'qa2a_1', 'qa2a_2', 'qa2a_3', 'qa2a_4', 'qa2a_5', 'qa2a_6', 'qa2a_7', 'qa7', 'qa8_1', 'qa8_2', 'qa8_3', 'qa8_4', 'qa8_5', 'qa8_6', 'qa8_7', 'qa8_8', 'qa8_9', 'qa8_10', 'qa8_11', 'qa8_12', 'qa8_13', 'qa10_1', 'qa10_2', 'qa10_3', 'qa10_4', 'qa10_5', 'qa10_6', 'qa17a', 'qa17b', 'qa18a_1', 'qa18a_2', 'qa18a_3', 'qa18a_4', 'qa18a_5', 'qa18a_6', 'qa19', 'qb3_1', 'qb3_2', 'qc2a_3', 'qc2a_4', 'qc2a_5', 'qd1a_1', 'qd1a_2', 'qd1a_3', 'qd1a_4', 'qd2_1', 'qd2_2', 'qd2_3', 'qd8', 'qd10a', 'qd10b', 'qd12_1', 'qd12_2', 'qe2_1', 'qe2_2', 'qe2_3', 'qe2_4', 'qg10', 'qg11_1', 'qg11_2', 'qg11_3', 'qg11_4', 'qg11_5', 'qg11_6', 'qg11_7', 'qg11_8', 'qg11_9', 'd1', 'd60', 'd62_1', 'd62_2', 'd63', 'd72_1', 'd72_2', 'd77']


In [89]:
# Drop DKs for ordinal variables

for i in range(86):
  eppes = eppes[eppes[col_names_ordinal[i]] != max(eppes[col_names_ordinal[i]])]


In [90]:
for i in range(86):
  print(eppes[col_names_ordinal[i]].value_counts())

2.0    5011
1.0    2322
3.0    1034
4.0     220
Name: d70, dtype: int64
2.0    5037
1.0    2273
3.0    1277
Name: d71a_1, dtype: int64
2.0    5122
3.0    1882
1.0    1583
Name: d71a_2, dtype: int64
2.0    5047
1.0    2087
3.0    1453
Name: d71a_3, dtype: int64
2.0    4584
3.0    2629
1.0     780
4.0     594
Name: qa1a_1, dtype: int64
2.0    3953
3.0    2821
1.0    1072
4.0     741
Name: qa1a_2, dtype: int64
2.0    5166
3.0    2335
1.0     770
4.0     316
Name: qa1a_3, dtype: int64
2.0    4619
1.0    1937
3.0    1599
4.0     432
Name: qa1a_4, dtype: int64
2.0    5120
3.0    1704
1.0    1430
4.0     333
Name: qa1a_5, dtype: int64
2.0    3927
3.0    2739
1.0    1048
4.0     873
Name: qa1a_6, dtype: int64
2.0    4293
3.0    2712
1.0     919
4.0     663
Name: qa1a_7, dtype: int64
3.0    4706
1.0    2938
2.0     943
Name: qa2a_1, dtype: int64
3.0    4408
1.0    2334
2.0    1845
Name: qa2a_2, dtype: int64
3.0    4262
1.0    2197
2.0    2128
Name: qa2a_3, dtype: int64
3.0    4911
1.0    2494
2

In [44]:
eppes.describe()

,studyno1,studyno2,edition,survey,caseid,uniqid,serialid,tnscntry,country,q1.1,q1.2,q1.3,q1.4,q1.5,q1.6,q1.7,q1.8,q1.9,q1.10,q1.11,q1.12,q1.13,q1.14,q1.15,q1.16,q1.17,q1.18,q1.19,q1.20,q1.21,q1.22,q1.23,q1.24,q1.25,q1.26,q1.27,q1.28,d70,d71a_1,d71a_2,d71a_3,qa1a_1,qa1a_2,qa1a_3,qa1a_4,qa1a_5,qa1a_6,qa1a_7,qa2a_1,qa2a_2,qa2a_3,qa2a_4,qa2a_5,qa2a_6,qa2a_7,qa3a.1,qa3a.2,qa3a.3,qa3a.4,qa3a.5,qa3a.6,qa3a.7,qa3a.8,qa3a.9,qa3a.10,qa3a.11,qa3a.12,qa3a.13,qa3a.14,qa3a.15,qa3a.16,qa4a.1,qa4a.2,qa4a.3,qa4a.4,qa4a.5,qa4a.6,qa4a.7,qa4a.8,qa4a.9,qa4a.10,qa4a.11,qa4a.12,qa4a.13,qa4a.14,qa4a.15,qa4a.16,qa4a.17,qa4a.18,qa5.1,qa5.2,qa5.3,qa5.4,qa5.5,qa5.6,qa5.7,qa5.8,qa5.9,qa5.10,qa5.11,qa5.12,qa5.13,qa5.14,qa5.15,qa5.16,d73a_1,d73a_2,d73a_3,qa6a_1,qa6a_2,qa6a_3,qa6a_4,qa6a_5,qa6a_6,qa6a_7,qa6a_8,qa6a_9,qa6a_10,qa6a_11,qa7,qa8_1,qa8_2,qa8_3,qa8_4,qa8_5,qa8_6,qa8_7,qa8_8,qa8_9,qa8_10,qa8_11,qa8_12,qa8_13,qa9.1,qa9.2,qa9.3,qa9.4,qa9.5,qa9.6,qa9.7,qa9.8,qa9.9,qa9.10,qa9.11,qa9.12,qa9.13,qa9.14,qa9.15,qa9.16,qa10_1,qa10_2,qa10_3,qa10_4,qa10_5,qa10_6,qf1,qf2,qa13_1,qa13_2,qa13_3,qa13_4,qa13_5,qa14_1,qa14_2,qa14_3,qa14_4,qa14_5,qa15_1,qa15_2,qa15_3,qa16_1,qa16_2,qa16_3,qa16_4,qa16_5,qa16_6,qa16_7,qa16_8,qa16_9,qa17a,qa17b,qa18a_1,qa18a_2,qa18a_3,qa18a_4,qa18a_5,qa18a_6,qa19,qb3_1,qb3_2,qb5_1,qb5_2,qb5_3,qb5_4,qc1,qc2a_3,qc2a_4,qc2a_5,qd1a_1,qd1a_2,qd1a_3,qd1a_4,qd2_1,qd2_2,qd2_3,qd3,qd4a,qd5.1,qd5.2,qd5.3,qd5.4,qd5.5,qd5.6,qd5.7,qd5.8,qd5.9,qd5.10,qd5.11,qd5.12,qd5.13,qd5.14,qd5.15,qd5.16,qd5.17,qd6.1,qd6.2,qd6.3,qd6.4,qd6.5,qd6.6,qd6.7,qd6.8,qd6.9,qd6.10,qd6.11,qd6.12,qd6.13,qd6.14,qd7.1,qd7.2,qd7.3,qd7.4,qd7.5,qd7.6,qd7.7,qd7.8,qd7.9,qd7.10,qd7.11,qd7.12,qd7.13,qd7.14,qd8,qd9_1,qd9_2,qd9_3,qd9_4,qd9_5,qd9_6,qd9_7,qd10a,qd10ar,qd10b,qd11_1,qd11_2,qd11_3,qd11_4,qd11_5,qd11_6,qd11_7,qd11_8,qd12_1,qd12_2,qe1_1,qe1_2,qe1_3,qe1_4,qe1_5,qe1_6,qe1_7,qe1_8,qe2_1,qe2_2,qe2_3,qe2_4,qg1,qg10,qg11_1,qg11_2,qg11_3,qg11_4,qg11_5,qg11_6,qg11_7,qg11_8,qg11_9,qg12,qg13_1,qg13_2,qg13_3,qg13_4,qg13_5,qg15,d1,d7,d11,d15a,d25,d40a,d40b,d43a,d43b,d46.8,d60,d62_1,d62_2,d63,d72_1,d72_2,d77
count,27464.0,27464.0,27464.0,27464.0,2.746400e+04,2.746400e+04,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.00000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.00000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.00000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,27464.000000,

In [45]:
# Data Exploration

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(eppes.dtypes)

studyno1    float64
studyno2    float64
doi          object
version      object
edition     float64
survey      float64
caseid      float64
uniqid      float64
serialid    float64
tnscntry    float64
country     float64
isocntry     object
q1.1        float64
q1.2        float64
q1.3        float64
q1.4        float64
q1.5        float64
q1.6        float64
q1.7        float64
q1.8        float64
q1.9        float64
q1.10       float64
q1.11       float64
q1.12       float64
q1.13       float64
q1.14       float64
q1.15       float64
q1.16       float64
q1.17       float64
q1.18       float64
q1.19       float64
q1.20       float64
q1.21       float64
q1.22       float64
q1.23       float64
q1.24       float64
q1.25       float64
q1.26       float64
q1.27       float64
q1.28       float64
d70         float64
d71a_1      float64
d71a_2      float64
d71a_3      float64
qa1a_1      float64
qa1a_2      float64
qa1a_3      float64
qa1a_4      float64
qa1a_5      float64
qa1a_6      float64


## Change Value type from float to object for categoricals

In [46]:
eppes.dtypes.value_counts()

float64    320
object       3
dtype: int64

In [99]:
# Create list of names of categorical columns 

col_names_categorical = []

for i in range(872):
  if columns_analysis.iloc[i, 3] == 'categorical':
    col_names_categorical.append(columns_analysis.iloc[i, 1])